# Bite-Sized Neo4j for Data Scientists
## Part 8: Importing data into the database from a JSON file

This video is about importing data from a properly-formatted JSON file into a Neo4j database.  The data comes from the [Kaggle 2020 Machine Learning and Data Science Survey](https://www.kaggle.com/c/kaggle-survey-2020).  There are 355 columns for this data, so I went ahead and selected a subset of those that I thought would be interesting.

The notebook just shows how I created the JSON file.  I ran this on my local machine (see the uri and password when I make the connection to Neo4j below) using the Docker container from [this blog post](https://dev.neo4j.com/docker_neo_jupyter) on Towards Data Science.  That container has both Neo4j and Jupyter so you can run and network them together and can be used for anything.  You don't actually need this bit since this video just goes over the data import.  I just wanted to provide it so you can see how I created the dataset.

Once the graph is populated below, I use 

```
CALL apoc.export.json.all("kaggle.json",{useTypes:true})
```

to export the graph to JSON, which puts it in the ` /var/lib/neo4j/import` directory in the Neo4j part of the container, which is then available through volume forwarding in `$HOME/graph_data/my_data`.

In [1]:
import pandas as pd
from neo4j import GraphDatabase

In [2]:
orig_df = pd.read_csv('./data/kaggle_survey_2020_responses.csv')
orig_df.head()

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,...,Q35_B_Part_2,Q35_B_Part_3,Q35_B_Part_4,Q35_B_Part_5,Q35_B_Part_6,Q35_B_Part_7,Q35_B_Part_8,Q35_B_Part_9,Q35_B_Part_10,Q35_B_OTHER
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,For how many years have you been writing code ...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,What programming languages do you use on a reg...,...,"In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor...","In the next 2 years, do you hope to become mor..."
1,1838,35-39,Man,Colombia,Doctoral degree,Student,5-10 years,Python,R,SQL,...,NaN,NaN,NaN,TensorBoard,NaN,NaN,NaN,NaN,NaN,NaN
2,289287,30-34,Man,United States of America,Master’s degree,Data Engineer,5-10 years,Python,R,SQL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,860,35-39,Man,Argentina,Bachelor’s degree,Software Engineer,10-20 years,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
4,507,30-34,Man,United States of America,Master’s degree,Data Scientist,5-10 years,Python,NaN,SQL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
orig_df.shape

(20037, 355)

In [4]:
for col in orig_df.columns:
    print(col)

Time from Start to Finish (seconds)
Q1
Q2
Q3
Q4
Q5
Q6
Q7_Part_1
Q7_Part_2
Q7_Part_3
Q7_Part_4
Q7_Part_5
Q7_Part_6
Q7_Part_7
Q7_Part_8
Q7_Part_9
Q7_Part_10
Q7_Part_11
Q7_Part_12
Q7_OTHER
Q8
Q9_Part_1
Q9_Part_2
Q9_Part_3
Q9_Part_4
Q9_Part_5
Q9_Part_6
Q9_Part_7
Q9_Part_8
Q9_Part_9
Q9_Part_10
Q9_Part_11
Q9_OTHER
Q10_Part_1
Q10_Part_2
Q10_Part_3
Q10_Part_4
Q10_Part_5
Q10_Part_6
Q10_Part_7
Q10_Part_8
Q10_Part_9
Q10_Part_10
Q10_Part_11
Q10_Part_12
Q10_Part_13
Q10_OTHER
Q11
Q12_Part_1
Q12_Part_2
Q12_Part_3
Q12_OTHER
Q13
Q14_Part_1
Q14_Part_2
Q14_Part_3
Q14_Part_4
Q14_Part_5
Q14_Part_6
Q14_Part_7
Q14_Part_8
Q14_Part_9
Q14_Part_10
Q14_Part_11
Q14_OTHER
Q15
Q16_Part_1
Q16_Part_2
Q16_Part_3
Q16_Part_4
Q16_Part_5
Q16_Part_6
Q16_Part_7
Q16_Part_8
Q16_Part_9
Q16_Part_10
Q16_Part_11
Q16_Part_12
Q16_Part_13
Q16_Part_14
Q16_Part_15
Q16_OTHER
Q17_Part_1
Q17_Part_2
Q17_Part_3
Q17_Part_4
Q17_Part_5
Q17_Part_6
Q17_Part_7
Q17_Part_8
Q17_Part_9
Q17_Part_10
Q17_Part_11
Q17_OTHER
Q18_Part_1
Q18_Part_2
Q18_Part_

## Select the columns we are going to use to populate the graph and put them in a usable format

Consult the Kaggle report to understand what each question means.

In [28]:
def drop_nan(line):

    return [x for x in line if str(x) != 'nan']


df = pd.DataFrame()

df['country'] = orig_df['Q3']
df['languages'] = orig_df[['Q7_Part_1', 'Q7_Part_2', 'Q7_Part_3', 'Q7_Part_4', 'Q7_Part_5',
                           'Q7_Part_6', 'Q7_Part_7', 'Q7_Part_8', 'Q7_Part_9', 'Q7_Part_10',
                           'Q7_Part_11', 'Q7_Part_12', 'Q7_OTHER']].values.tolist()

df['databases'] = orig_df[['Q29_B_Part_1', 'Q29_B_Part_2', 'Q29_B_Part_3', 'Q29_B_Part_4',
                           'Q29_B_Part_5', 'Q29_B_Part_6', 'Q29_B_Part_7', 'Q29_B_Part_8',
                           'Q29_B_Part_9', 'Q29_B_Part_10', 'Q29_B_Part_11', 'Q29_B_Part_12',
                           'Q29_B_Part_13', 'Q29_B_Part_14', 'Q29_B_Part_15', 'Q29_B_Part_16',
                           'Q29_B_Part_17', 'Q29_B_OTHER']].values.tolist()

df['auto_ml'] = orig_df[['Q34_B_Part_1', 'Q34_B_Part_2', 'Q34_B_Part_3', 'Q34_B_Part_4',
                         'Q34_B_Part_5', 'Q34_B_Part_6', 'Q34_B_Part_7', 'Q34_B_Part_8',
                         'Q34_B_Part_9', 'Q34_B_Part_10', 'Q34_B_Part_11', 'Q34_B_OTHER']].values.tolist()

df['frameworks'] = orig_df[['Q16_Part_1', 'Q16_Part_2', 'Q16_Part_3', 'Q16_Part_4',
                            'Q16_Part_5', 'Q16_Part_6', 'Q16_Part_7', 'Q16_Part_8',
                            'Q16_Part_9', 'Q16_Part_10', 'Q16_Part_11', 'Q16_Part_12',
                            'Q16_Part_13', 'Q16_Part_14', 'Q16_Part_15', 'Q16_OTHER']].values.tolist()


df['ml_products'] = orig_df[['Q28_A_Part_1', 'Q28_A_Part_2', 'Q28_A_Part_3', 'Q28_A_Part_4',
                             'Q28_A_Part_5', 'Q28_A_Part_6', 'Q28_A_Part_7', 'Q28_A_Part_8',
                             'Q28_A_Part_9', 'Q28_A_Part_10', 'Q28_A_OTHER']].values.tolist()

df['viz_tools'] = orig_df[['Q14_Part_1', 'Q14_Part_2', 'Q14_Part_3', 'Q14_Part_4',
                           'Q14_Part_5', 'Q14_Part_6', 'Q14_Part_7', 'Q14_Part_8',
                           'Q14_Part_9', 'Q14_Part_10', 'Q14_Part_11', 'Q14_OTHER']].values.tolist()

df['social_media'] = orig_df[['Q39_Part_1', 'Q39_Part_2', 'Q39_Part_3', 'Q39_Part_4',
                              'Q39_Part_5', 'Q39_Part_6', 'Q39_Part_7', 'Q39_Part_8',
                              'Q39_Part_9', 'Q39_Part_10', 'Q39_Part_11', 'Q39_OTHER']].values.tolist()

df['current_role'] = orig_df[['Q5']]
df['num_years_coding'] = orig_df[['Q6']]
df['num_data_scientist_team'] = orig_df['Q21']
df['ml_budget'] = orig_df[['Q25']]
df['recommended_language'] = orig_df[['Q8']]
df['most_used_database'] = orig_df[['Q30']]

df = df.loc[1:, :]

df['clean_languages'] = df['languages'].map(drop_nan)
df['clean_databases'] = df['databases'].map(drop_nan)
df['clean_auto_ml'] = df['auto_ml'].map(drop_nan)
df['clean_frameworks'] = df['frameworks'].map(drop_nan)
df['clean_ml_products'] = df['ml_products'].map(drop_nan)
df['clean_viz_tools'] = df['viz_tools'].map(drop_nan)
df['clean_social_media'] = df['social_media'].map(drop_nan)

df2 = df.drop(['languages'], axis=1)
df2 = df2.drop(['databases', 'auto_ml', 'frameworks', 'ml_products', 'viz_tools', 'social_media'], axis=1)
df2 = df2.reset_index()
df2.columns = ['id', 'country', 'current_role', 'num_years_coding', 
               'num_data_scientist_team', 'ml_budget', 'recommended_language', 'most_used_database', 
               'languages', 'databases', 'auto_ml', 'frameworks', 
               'ml_products', 'viz_tools', 'social_media']
df2.head()

,id,country,current_role,num_years_coding,num_data_scientist_team,ml_budget,recommended_language,most_used_database,languages,databases,auto_ml,frameworks,ml_products,viz_tools,social_media
0,1,Colombia,Student,5-10 years,NaN,NaN,Python,NaN,"[Python, R, SQL, C, Javascript, MATLAB, Other]","[MongoDB , Microsoft SQL Server , Google Cloud...","[ Google Cloud AutoML , Databricks AutoML , ...","[ TensorFlow , Keras , Xgboost ]",[],"[ Matplotlib , Geoplotlib ]","[Kaggle (notebooks, forums, etc), Journal Publ..."
1,2,United States of America,Data Engineer,5-10 years,20+,"$100,000 or more ($USD)",Python,PostgresSQL,"[Python, R, SQL]",[],[],"[ Scikit-learn , TensorFlow , Keras , PyT...",[ Amazon SageMaker ],"[ Matplotlib , Seaborn , Ggplot / ggplot2 , ...","[Twitter (data science influencers), Reddit (r..."
2,3,Argentina,Software Engineer,10-20 years,0,$0 ($USD),R,NaN,"[Java, Javascript, Bash]","[MySQL , Microsoft SQL Server ]",[],[],[],[ D3 js ],"[Email newsletters (Data Elixir, O'Reilly Data..."
3,4,United States of America,Data Scientist,5-10 years,5-9,"$10,000-$99,999",Python,MySQL,"[Python, SQL, Bash]",[],[],"[ Scikit-learn , TensorFlow , Keras , Xgb...",[No / None],"[ Matplotlib , Seaborn , Plotly / Plotly Exp...",[None]
4,5,Japan,Software Engineer,3-5 years,NaN,NaN,Python,NaN,[Python],[],[],[],[],[],[]


In [6]:
from neo4j import GraphDatabase

In [7]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

## This is what the connection looks like when going through Docker 

Note the user name and password are specified in the `docker-compose.yml` file.

In [8]:
conn = Neo4jConnection(uri="bolt://neo4j:7687", user="neo4j", pwd="1234")

In [9]:
conn.query('CREATE CONSTRAINT ids IF NOT EXISTS ON (i:Id) ASSERT i.id IS UNIQUE')
conn.query('CREATE CONSTRAINT languages IF NOT EXISTS ON (l:Language) ASSERT l.name IS UNIQUE')
conn.query('CREATE CONSTRAINT databases IF NOT EXISTS ON (d:Database) ASSERT d.name IS UNIQUE')
conn.query('CREATE CONSTRAINT automl IF NOT EXISTS ON (a:AutoMl) ASSERT a.name IS UNIQUE')
conn.query('CREATE CONSTRAINT frameworks IF NOT EXISTS ON (f:Framework) ASSERT f.name IS UNIQUE')
conn.query('CREATE CONSTRAINT products IF NOT EXISTS ON (p:MlProduct) ASSERT p.name IS UNIQUE')
conn.query('CREATE CONSTRAINT viztools IF NOT EXISTS ON (v:VizTool) ASSERT v.name IS UNIQUE')
conn.query('CREATE CONSTRAINT socialmedia IF NOT EXISTS ON (s:SocialMedia) ASSERT s.name IS UNIQUE')

[]

## These are a whole bunch of functions that take parts of the dataframe and turn them into nodes and relationships within the graph

In [10]:
def add_ids(rows, batch_size=1000):
    # Adds author nodes to the Neo4j graph as a batch job.

    query = '''UNWIND $rows AS row
               MERGE (:Id {name: row.id, country: row.country, current_role: row.current_role, num_years_coding: row.num_years_coding, num_data_scientist_team: row.num_data_scientist_team, ml_budget: row.ml_budget})
               RETURN count(*) as total
    '''
    return insert_data(query, rows, batch_size)


def add_langs(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               UNWIND row.languages AS language
               MERGE (:Language {name: language})
               RETURN COUNT(*) AS total
    """
    
    return insert_data(query, rows, batch_size)


def add_dbs(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               UNWIND row.databases AS database
               MERGE (:Database {name: database})
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_auto_ml(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               UNWIND row.auto_ml AS auto_ml
               MERGE (:AutoMl {name: auto_ml})
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_frameworks(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               UNWIND row.frameworks AS framework
               MERGE (:Framework {name: framework})
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_products(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               UNWIND row.ml_products AS product
               MERGE (:MlProduct {name: product})
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_viz_tools(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               UNWIND row.viz_tools AS viz_tool
               MERGE (:VizTool {name: viz_tool})
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_social_media(rows, batch_size=1000):

    query = """UNWIND $rows AS row
               UNWIND row.social_media AS social_media
               MERGE (:SocialMedia {name: social_media})
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_all_lang_rels(rows, batch_size=500):
    # Adds paper nodes and (:Author)--(:Paper) and (:Paper)--(:Category)
    # relationships to the Neo4j graph as a batch job.  (Note the smaller batch
    # size due to the fact that this function is adding much more data than the
    # add_authors() function.)

    query = """UNWIND $rows AS row
               UNWIND row.languages AS language
               MATCH (i:Id {name: row.id})
               MATCH (l:Language {name: language})
               MERGE (i)-[:USES_LANGUAGE]->(l)
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_rec_lang_rel(rows, batch_size=500):

    query = """UNWIND $rows AS row
               MATCH (i:Id {name: row.id})
               MATCH (l:Language {name: row.recommended_language})
               MERGE (i)-[:RECOMMENDS_LANGUAGE]->(l)
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_all_db_rels(rows, batch_size=500):

    query = """UNWIND $rows AS row
               UNWIND row.databases AS database
               MATCH (i:Id {name: row.id})
               MATCH (d:Database {name: database})
               MERGE (i)-[:USES_DATABASE]->(d)
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_most_used_db_rel(rows, batch_size=500):

    query = """UNWIND $rows AS row
               MATCH (i:Id {name: row.id})
               MATCH (d:Database {name: row.most_used_database})
               MERGE (i)-[:MOST_OFTEN_USED_DB]->(d)
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_auto_ml_rels(rows, batch_size=500):

    query = """UNWIND $rows AS row
               UNWIND row.auto_ml AS auto_ml
               MATCH (i:Id {name: row.id})
               MATCH (a:AutoMl {name: auto_ml})
               MERGE (i)-[:USES_AUTO_ML]->(a)
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_framework_rels(rows, batch_size=500):

    query = """UNWIND $rows AS row
               UNWIND row.frameworks AS framework
               MATCH (i:Id {name: row.id})
               MATCH (f:Framework {name: framework})
               MERGE (i)-[:USES_FRAMEWORK]->(f)
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_product_rels(rows, batch_size=500):

    query = """UNWIND $rows AS row
               UNWIND row.ml_products AS product
               MATCH (i:Id {name: row.id})
               MATCH (p:MlProduct {name: product})
               MERGE (i)-[:USES_ML_PROD]->(p)
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_viz_tool_rels(rows, batch_size=500):

    query = """UNWIND $rows AS row
               UNWIND row.viz_tools AS viz_tool
               MATCH (i:Id {name: row.id})
               MATCH (v:VizTool {name: viz_tool})
               MERGE (i)-[:USES_VIZ_TOOL]->(v)
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def add_social_media_rels(rows, batch_size=500):

    query = """UNWIND $rows AS row
               UNWIND row.social_media AS social_media
               MATCH (i:Id {name: row.id})
               MATCH (s:SocialMedia {name: social_media})
               MERGE (i)-[:USES_SOCIAL_MEDIA]->(s)
               RETURN COUNT(*) AS total
    """

    return insert_data(query, rows, batch_size)


def insert_data(query, rows, batch_size = 10000):
    # Function to handle the updating the Neo4j database in batch mode.

    total = 0
    batch = 0
    start = time.time()
    result = None

    while batch * batch_size < len(rows):

        res = conn.query(query, parameters={'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, "batches":batch, "time":time.time()-start}
        print(result)

    return result

In [11]:
add_ids(df2)

{'total': 1000, 'batches': 1, 'time': 0.962932825088501}
{'total': 2000, 'batches': 2, 'time': 2.0747017860412598}
{'total': 3000, 'batches': 3, 'time': 3.638833522796631}
{'total': 4000, 'batches': 4, 'time': 5.761361837387085}
{'total': 5000, 'batches': 5, 'time': 8.35218620300293}
{'total': 6000, 'batches': 6, 'time': 11.466829299926758}
{'total': 7000, 'batches': 7, 'time': 15.179715871810913}
{'total': 8000, 'batches': 8, 'time': 19.41966438293457}
{'total': 9000, 'batches': 9, 'time': 24.212233781814575}
{'total': 10000, 'batches': 10, 'time': 29.517643690109253}
{'total': 11000, 'batches': 11, 'time': 34.46762037277222}
{'total': 12000, 'batches': 12, 'time': 39.85889005661011}
{'total': 13000, 'batches': 13, 'time': 45.58579659461975}
{'total': 14000, 'batches': 14, 'time': 51.62421679496765}
{'total': 15000, 'batches': 15, 'time': 58.157466888427734}
{'total': 16000, 'batches': 16, 'time': 65.10491633415222}
{'total': 17000, 'batches': 17, 'time': 72.47278237342834}
{'total': 

{'total': 20036, 'batches': 21, 'time': 97.69752478599548}

In [12]:
add_langs(df2)

{'total': 2291, 'batches': 1, 'time': 0.3669300079345703}
{'total': 4621, 'batches': 2, 'time': 0.6144332885742188}
{'total': 6992, 'batches': 3, 'time': 0.8470773696899414}
{'total': 9290, 'batches': 4, 'time': 1.0902841091156006}
{'total': 11639, 'batches': 5, 'time': 1.3264706134796143}
{'total': 14003, 'batches': 6, 'time': 1.5995230674743652}
{'total': 16376, 'batches': 7, 'time': 1.827732801437378}
{'total': 18806, 'batches': 8, 'time': 2.047433853149414}
{'total': 21146, 'batches': 9, 'time': 2.2674665451049805}
{'total': 23440, 'batches': 10, 'time': 2.4922542572021484}
{'total': 25801, 'batches': 11, 'time': 2.716212511062622}
{'total': 28249, 'batches': 12, 'time': 2.9377639293670654}
{'total': 30672, 'batches': 13, 'time': 3.157878875732422}
{'total': 33107, 'batches': 14, 'time': 3.377192497253418}
{'total': 35438, 'batches': 15, 'time': 3.6151981353759766}
{'total': 37787, 'batches': 16, 'time': 3.857487916946411}
{'total': 40192, 'batches': 17, 'time': 4.087592124938965}


{'total': 47450, 'batches': 21, 'time': 4.797094821929932}

In [13]:
add_all_lang_rels(df2)

{'total': 1159, 'batches': 1, 'time': 11.89229679107666}
{'total': 2291, 'batches': 2, 'time': 23.27991819381714}
{'total': 3461, 'batches': 3, 'time': 34.94558382034302}
{'total': 4621, 'batches': 4, 'time': 46.61304759979248}
{'total': 5792, 'batches': 5, 'time': 58.486855268478394}
{'total': 6992, 'batches': 6, 'time': 70.42991471290588}
{'total': 8166, 'batches': 7, 'time': 82.17660880088806}
{'total': 9290, 'batches': 8, 'time': 93.43988561630249}
{'total': 10466, 'batches': 9, 'time': 105.22930908203125}
{'total': 11639, 'batches': 10, 'time': 117.05418014526367}
{'total': 12803, 'batches': 11, 'time': 128.0705099105835}
{'total': 14003, 'batches': 12, 'time': 139.32308197021484}
{'total': 15178, 'batches': 13, 'time': 150.3031027317047}
{'total': 16376, 'batches': 14, 'time': 161.52330136299133}
{'total': 17614, 'batches': 15, 'time': 173.05547523498535}
{'total': 18806, 'batches': 16, 'time': 184.19707560539246}
{'total': 19954, 'batches': 17, 'time': 194.90309500694275}
{'tota

{'total': 47450, 'batches': 41, 'time': 450.88008856773376}

In [14]:
add_rec_lang_rel(df2)

{'total': 443, 'batches': 1, 'time': 5.1144726276397705}
{'total': 892, 'batches': 2, 'time': 10.160549402236938}
{'total': 1339, 'batches': 3, 'time': 15.255979776382446}
{'total': 1779, 'batches': 4, 'time': 20.316847801208496}
{'total': 2216, 'batches': 5, 'time': 25.4208881855011}
{'total': 2659, 'batches': 6, 'time': 30.4552800655365}
{'total': 3097, 'batches': 7, 'time': 35.496543407440186}
{'total': 3536, 'batches': 8, 'time': 40.48268532752991}
{'total': 3980, 'batches': 9, 'time': 45.526402950286865}
{'total': 4422, 'batches': 10, 'time': 50.66824412345886}
{'total': 4870, 'batches': 11, 'time': 55.37342429161072}
{'total': 5312, 'batches': 12, 'time': 60.115267753601074}
{'total': 5757, 'batches': 13, 'time': 64.79586911201477}
{'total': 6194, 'batches': 14, 'time': 69.53296494483948}
{'total': 6644, 'batches': 15, 'time': 74.20875787734985}
{'total': 7091, 'batches': 16, 'time': 78.8406434059143}
{'total': 7541, 'batches': 17, 'time': 83.5461106300354}
{'total': 7982, 'batch

{'total': 17821, 'batches': 41, 'time': 193.60888409614563}

In [15]:
add_dbs(df2)

{'total': 1247, 'batches': 1, 'time': 0.256925106048584}
{'total': 2700, 'batches': 2, 'time': 0.5005044937133789}
{'total': 3974, 'batches': 3, 'time': 0.7249860763549805}
{'total': 5399, 'batches': 4, 'time': 0.9405431747436523}
{'total': 6775, 'batches': 5, 'time': 1.1606030464172363}
{'total': 8074, 'batches': 6, 'time': 1.3782927989959717}
{'total': 9430, 'batches': 7, 'time': 1.5980761051177979}
{'total': 10887, 'batches': 8, 'time': 1.8158488273620605}
{'total': 12334, 'batches': 9, 'time': 2.036309003829956}
{'total': 13631, 'batches': 10, 'time': 2.261573553085327}
{'total': 15124, 'batches': 11, 'time': 2.49010968208313}
{'total': 16458, 'batches': 12, 'time': 2.712306022644043}
{'total': 17839, 'batches': 13, 'time': 2.933023691177368}
{'total': 19199, 'batches': 14, 'time': 3.1478054523468018}
{'total': 20491, 'batches': 15, 'time': 3.3608572483062744}
{'total': 21983, 'batches': 16, 'time': 3.5775535106658936}
{'total': 23532, 'batches': 17, 'time': 3.8021605014801025}
{'t

{'total': 27373, 'batches': 21, 'time': 4.546523094177246}

In [16]:
add_all_db_rels(df2)

{'total': 686, 'batches': 1, 'time': 6.955230474472046}
{'total': 1247, 'batches': 2, 'time': 12.669395446777344}
{'total': 2010, 'batches': 3, 'time': 20.498956203460693}
{'total': 2700, 'batches': 4, 'time': 27.392053842544556}
{'total': 3333, 'batches': 5, 'time': 33.78075408935547}
{'total': 3974, 'batches': 6, 'time': 40.195393562316895}
{'total': 4776, 'batches': 7, 'time': 48.24994683265686}
{'total': 5399, 'batches': 8, 'time': 54.54207420349121}
{'total': 6027, 'batches': 9, 'time': 60.865509033203125}
{'total': 6775, 'batches': 10, 'time': 68.38058733940125}
{'total': 7435, 'batches': 11, 'time': 74.60566210746765}
{'total': 8074, 'batches': 12, 'time': 80.60880899429321}
{'total': 8728, 'batches': 13, 'time': 86.76592469215393}
{'total': 9430, 'batches': 14, 'time': 93.35770153999329}
{'total': 10223, 'batches': 15, 'time': 100.77417993545532}
{'total': 10887, 'batches': 16, 'time': 107.00645613670349}
{'total': 11555, 'batches': 17, 'time': 113.32725834846497}
{'total': 123

{'total': 27373, 'batches': 41, 'time': 261.0800607204437}

In [17]:
add_most_used_db_rel(df2)

{'total': 78, 'batches': 1, 'time': 5.1052565574646}
{'total': 173, 'batches': 2, 'time': 10.214683055877686}
{'total': 264, 'batches': 3, 'time': 15.30597710609436}
{'total': 348, 'batches': 4, 'time': 20.380758047103882}
{'total': 427, 'batches': 5, 'time': 25.46423077583313}
{'total': 529, 'batches': 6, 'time': 30.59005618095398}
{'total': 600, 'batches': 7, 'time': 35.609539270401}
{'total': 680, 'batches': 8, 'time': 40.834779024124146}
{'total': 772, 'batches': 9, 'time': 45.91334795951843}
{'total': 841, 'batches': 10, 'time': 50.97310829162598}
{'total': 933, 'batches': 11, 'time': 55.695130348205566}
{'total': 1027, 'batches': 12, 'time': 60.38695573806763}
{'total': 1116, 'batches': 13, 'time': 65.07891774177551}
{'total': 1194, 'batches': 14, 'time': 69.81154537200928}
{'total': 1273, 'batches': 15, 'time': 74.57708525657654}
{'total': 1359, 'batches': 16, 'time': 79.37392091751099}
{'total': 1454, 'batches': 17, 'time': 84.24772310256958}
{'total': 1540, 'batches': 18, 'tim

{'total': 3513, 'batches': 41, 'time': 193.5530321598053}

In [18]:
add_auto_ml(df2)

{'total': 595, 'batches': 1, 'time': 0.23352670669555664}
{'total': 1379, 'batches': 2, 'time': 0.4445183277130127}
{'total': 2061, 'batches': 3, 'time': 0.6564750671386719}
{'total': 2779, 'batches': 4, 'time': 0.8661506175994873}
{'total': 3455, 'batches': 5, 'time': 1.075575590133667}
{'total': 4171, 'batches': 6, 'time': 1.2876112461090088}
{'total': 4930, 'batches': 7, 'time': 1.5002520084381104}
{'total': 5633, 'batches': 8, 'time': 1.7160871028900146}
{'total': 6405, 'batches': 9, 'time': 1.9316456317901611}
{'total': 7065, 'batches': 10, 'time': 2.142219066619873}
{'total': 7763, 'batches': 11, 'time': 2.3596107959747314}
{'total': 8488, 'batches': 12, 'time': 2.573195695877075}
{'total': 9128, 'batches': 13, 'time': 2.7900705337524414}
{'total': 9812, 'batches': 14, 'time': 3.009068489074707}
{'total': 10455, 'batches': 15, 'time': 3.227391242980957}
{'total': 11147, 'batches': 16, 'time': 3.4374685287475586}
{'total': 11927, 'batches': 17, 'time': 3.651763439178467}
{'total':

{'total': 13921, 'batches': 21, 'time': 4.363068103790283}

In [19]:
add_auto_ml_rels(df2)

{'total': 335, 'batches': 1, 'time': 3.5041697025299072}
{'total': 595, 'batches': 2, 'time': 6.217179775238037}
{'total': 1054, 'batches': 3, 'time': 10.9210045337677}
{'total': 1379, 'batches': 4, 'time': 14.29832148551941}
{'total': 1729, 'batches': 5, 'time': 17.915026426315308}
{'total': 2061, 'batches': 6, 'time': 21.33316659927368}
{'total': 2459, 'batches': 7, 'time': 25.371464729309082}
{'total': 2779, 'batches': 8, 'time': 28.658487796783447}
{'total': 3078, 'batches': 9, 'time': 31.75278902053833}
{'total': 3455, 'batches': 10, 'time': 35.62026023864746}
{'total': 3821, 'batches': 11, 'time': 39.15197682380676}
{'total': 4171, 'batches': 12, 'time': 42.483396768569946}
{'total': 4530, 'batches': 13, 'time': 45.91202521324158}
{'total': 4930, 'batches': 14, 'time': 49.771788120269775}
{'total': 5313, 'batches': 15, 'time': 53.44936752319336}
{'total': 5633, 'batches': 16, 'time': 56.55524182319641}
{'total': 6003, 'batches': 17, 'time': 60.110421895980835}
{'total': 6405, 'ba

{'total': 13921, 'batches': 41, 'time': 136.1004753112793}

In [20]:
add_frameworks(df2)

{'total': 1956, 'batches': 1, 'time': 0.24299407005310059}
{'total': 4002, 'batches': 2, 'time': 0.4648299217224121}
{'total': 5987, 'batches': 3, 'time': 0.6821470260620117}
{'total': 7933, 'batches': 4, 'time': 0.9075586795806885}
{'total': 9852, 'batches': 5, 'time': 1.1882734298706055}
{'total': 11875, 'batches': 6, 'time': 1.4042229652404785}
{'total': 13873, 'batches': 7, 'time': 1.6188857555389404}
{'total': 15835, 'batches': 8, 'time': 1.8356127738952637}
{'total': 17856, 'batches': 9, 'time': 2.050997734069824}
{'total': 19776, 'batches': 10, 'time': 2.2648653984069824}
{'total': 21790, 'batches': 11, 'time': 2.484682321548462}
{'total': 23820, 'batches': 12, 'time': 2.7018654346466064}
{'total': 25733, 'batches': 13, 'time': 2.9193856716156006}
{'total': 27649, 'batches': 14, 'time': 3.1393206119537354}
{'total': 29577, 'batches': 15, 'time': 3.3605055809020996}
{'total': 31411, 'batches': 16, 'time': 3.57831072807312}
{'total': 33441, 'batches': 17, 'time': 3.802963018417358

{'total': 39178, 'batches': 21, 'time': 4.4662933349609375}

In [21]:
add_framework_rels(df2)

{'total': 994, 'batches': 1, 'time': 10.139696598052979}
{'total': 1956, 'batches': 2, 'time': 19.899624824523926}
{'total': 2996, 'batches': 3, 'time': 30.1230788230896}
{'total': 4002, 'batches': 4, 'time': 40.13197994232178}
{'total': 4957, 'batches': 5, 'time': 49.63934540748596}
{'total': 5987, 'batches': 6, 'time': 59.84809374809265}
{'total': 6975, 'batches': 7, 'time': 69.67918515205383}
{'total': 7933, 'batches': 8, 'time': 79.23358869552612}
{'total': 8906, 'batches': 9, 'time': 88.78422737121582}
{'total': 9852, 'batches': 10, 'time': 98.12023448944092}
{'total': 10880, 'batches': 11, 'time': 108.27761769294739}
{'total': 11875, 'batches': 12, 'time': 118.09820914268494}
{'total': 12848, 'batches': 13, 'time': 127.64952445030212}
{'total': 13873, 'batches': 14, 'time': 137.636385679245}
{'total': 14895, 'batches': 15, 'time': 146.95790886878967}
{'total': 15835, 'batches': 16, 'time': 155.53625583648682}
{'total': 16888, 'batches': 17, 'time': 165.22629117965698}
{'total': 1

{'total': 39178, 'batches': 41, 'time': 369.6187958717346}

In [22]:
add_products(df2)

{'total': 296, 'batches': 1, 'time': 0.23112249374389648}
{'total': 616, 'batches': 2, 'time': 0.4422037601470947}
{'total': 927, 'batches': 3, 'time': 0.6494994163513184}
{'total': 1216, 'batches': 4, 'time': 0.8597822189331055}
{'total': 1500, 'batches': 5, 'time': 1.0659313201904297}
{'total': 1796, 'batches': 6, 'time': 1.27449369430542}
{'total': 2118, 'batches': 7, 'time': 1.486138105392456}
{'total': 2422, 'batches': 8, 'time': 1.698072910308838}
{'total': 2769, 'batches': 9, 'time': 1.9070448875427246}
{'total': 3095, 'batches': 10, 'time': 2.1181142330169678}
{'total': 3424, 'batches': 11, 'time': 2.3303258419036865}
{'total': 3742, 'batches': 12, 'time': 2.542888641357422}
{'total': 4048, 'batches': 13, 'time': 2.7523300647735596}
{'total': 4365, 'batches': 14, 'time': 2.9630370140075684}
{'total': 4688, 'batches': 15, 'time': 3.2319133281707764}
{'total': 5036, 'batches': 16, 'time': 3.4414279460906982}
{'total': 5366, 'batches': 17, 'time': 3.6547045707702637}
{'total': 570

{'total': 6335, 'batches': 21, 'time': 4.3083577156066895}

In [23]:
add_product_rels(df2)

{'total': 124, 'batches': 1, 'time': 1.3461270332336426}
{'total': 296, 'batches': 2, 'time': 3.135993480682373}
{'total': 459, 'batches': 3, 'time': 4.8466339111328125}
{'total': 616, 'batches': 4, 'time': 6.493858814239502}
{'total': 750, 'batches': 5, 'time': 7.91748571395874}
{'total': 927, 'batches': 6, 'time': 9.772044897079468}
{'total': 1070, 'batches': 7, 'time': 11.282859563827515}
{'total': 1216, 'batches': 8, 'time': 12.81722116470337}
{'total': 1371, 'batches': 9, 'time': 14.442493915557861}
{'total': 1500, 'batches': 10, 'time': 15.82554316520691}
{'total': 1650, 'batches': 11, 'time': 17.30285143852234}
{'total': 1796, 'batches': 12, 'time': 18.746761560440063}
{'total': 1941, 'batches': 13, 'time': 20.175037145614624}
{'total': 2118, 'batches': 14, 'time': 21.887665033340454}
{'total': 2268, 'batches': 15, 'time': 23.370591402053833}
{'total': 2422, 'batches': 16, 'time': 24.87893772125244}
{'total': 2596, 'batches': 17, 'time': 26.56310486793518}
{'total': 2769, 'batch

{'total': 6335, 'batches': 41, 'time': 62.918365716934204}

In [24]:
add_viz_tools(df2)

{'total': 1810, 'batches': 1, 'time': 0.23310542106628418}
{'total': 3656, 'batches': 2, 'time': 0.4524707794189453}
{'total': 5504, 'batches': 3, 'time': 0.6642856597900391}
{'total': 7276, 'batches': 4, 'time': 0.874387264251709}
{'total': 9027, 'batches': 5, 'time': 1.0867154598236084}
{'total': 10855, 'batches': 6, 'time': 1.2995374202728271}
{'total': 12617, 'batches': 7, 'time': 1.5122778415679932}
{'total': 14461, 'batches': 8, 'time': 1.7280378341674805}
{'total': 16306, 'batches': 9, 'time': 2.001425266265869}
{'total': 18137, 'batches': 10, 'time': 2.216510057449341}
{'total': 20001, 'batches': 11, 'time': 2.4364709854125977}
{'total': 21868, 'batches': 12, 'time': 2.6520538330078125}
{'total': 23687, 'batches': 13, 'time': 2.862121343612671}
{'total': 25442, 'batches': 14, 'time': 3.0804665088653564}
{'total': 27237, 'batches': 15, 'time': 3.2957308292388916}
{'total': 29028, 'batches': 16, 'time': 3.5095276832580566}
{'total': 30927, 'batches': 17, 'time': 3.722288131713867

{'total': 36440, 'batches': 21, 'time': 4.371080636978149}

In [25]:
add_viz_tool_rels(df2)

{'total': 901, 'batches': 1, 'time': 8.83234691619873}
{'total': 1810, 'batches': 2, 'time': 17.690585136413574}
{'total': 2740, 'batches': 3, 'time': 26.763420343399048}
{'total': 3656, 'batches': 4, 'time': 35.73404002189636}
{'total': 4562, 'batches': 5, 'time': 44.62429642677307}
{'total': 5504, 'batches': 6, 'time': 53.80964970588684}
{'total': 6402, 'batches': 7, 'time': 62.637280225753784}
{'total': 7276, 'batches': 8, 'time': 71.19492769241333}
{'total': 8170, 'batches': 9, 'time': 79.95018005371094}
{'total': 9027, 'batches': 10, 'time': 88.3766303062439}
{'total': 9969, 'batches': 11, 'time': 96.94863653182983}
{'total': 10855, 'batches': 12, 'time': 105.00119614601135}
{'total': 11752, 'batches': 13, 'time': 113.21207642555237}
{'total': 12617, 'batches': 14, 'time': 121.09486126899719}
{'total': 13556, 'batches': 15, 'time': 129.65808582305908}
{'total': 14461, 'batches': 16, 'time': 137.9289743900299}
{'total': 15407, 'batches': 17, 'time': 146.5187304019928}
{'total': 163

{'total': 36440, 'batches': 41, 'time': 337.8044810295105}

In [26]:
add_social_media(df2)

{'total': 1908, 'batches': 1, 'time': 0.2455582618713379}
{'total': 3805, 'batches': 2, 'time': 0.4619781970977783}
{'total': 5743, 'batches': 3, 'time': 0.6824765205383301}
{'total': 7598, 'batches': 4, 'time': 0.8994946479797363}
{'total': 9454, 'batches': 5, 'time': 1.1175293922424316}
{'total': 11363, 'batches': 6, 'time': 1.3337392807006836}
{'total': 13267, 'batches': 7, 'time': 1.5486805438995361}
{'total': 15162, 'batches': 8, 'time': 1.7630622386932373}
{'total': 17021, 'batches': 9, 'time': 1.9781427383422852}
{'total': 18962, 'batches': 10, 'time': 2.19118070602417}
{'total': 20898, 'batches': 11, 'time': 2.40975284576416}
{'total': 22887, 'batches': 12, 'time': 2.624337911605835}
{'total': 24739, 'batches': 13, 'time': 2.8388161659240723}
{'total': 26568, 'batches': 14, 'time': 3.0576252937316895}
{'total': 28409, 'batches': 15, 'time': 3.3332927227020264}
{'total': 30258, 'batches': 16, 'time': 3.5432910919189453}
{'total': 32276, 'batches': 17, 'time': 3.7607030868530273}

{'total': 37937, 'batches': 21, 'time': 4.41117525100708}

In [27]:
add_social_media_rels(df2)

{'total': 909, 'batches': 1, 'time': 9.051838159561157}
{'total': 1908, 'batches': 2, 'time': 18.879411458969116}
{'total': 2915, 'batches': 3, 'time': 28.764257669448853}
{'total': 3805, 'batches': 4, 'time': 37.54251670837402}
{'total': 4716, 'batches': 5, 'time': 46.53784775733948}
{'total': 5743, 'batches': 6, 'time': 56.571343421936035}
{'total': 6698, 'batches': 7, 'time': 65.93193101882935}
{'total': 7598, 'batches': 8, 'time': 74.7197790145874}
{'total': 8531, 'batches': 9, 'time': 83.82578921318054}
{'total': 9454, 'batches': 10, 'time': 92.846839427948}
{'total': 10396, 'batches': 11, 'time': 101.41671395301819}
{'total': 11363, 'batches': 12, 'time': 110.2259840965271}
{'total': 12263, 'batches': 13, 'time': 118.45732712745667}
{'total': 13267, 'batches': 14, 'time': 127.56172060966492}
{'total': 14231, 'batches': 15, 'time': 136.3156669139862}
{'total': 15162, 'batches': 16, 'time': 144.7818365097046}
{'total': 16087, 'batches': 17, 'time': 153.26465034484863}
{'total': 170

{'total': 37937, 'batches': 41, 'time': 352.3045711517334}

## The number of nodes and relationships in the graph

In [29]:
conn.query("MATCH (n) RETURN COUNT(n)")

[<Record COUNT(n)=20130>]

In [30]:
conn.query("MATCH ()-[r]->() RETURN COUNT(r)")

[<Record COUNT(r)=229968>]